<a href="https://colab.research.google.com/github/ytyimin/scm518/blob/main/Motorized_Carrier_Selection_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Motorized Carrier Selection at Westvaco

## Objective and Prerequisites

This carrier assignment problem shows you how to determine the optimal assignment of carrier, while minimizing the overall cost. The objectives of the assignment problem are:

* Minimize the overall cost assignment as trucks,
* Make sure the number of trips are satisified, and
* Ensure that the assignments are valid, i.e., cannot assign a carrier to a route if the route does not exist.


---
## Problem Description

![picture](https://drive.google.com/uc?id=1gEj38xWby10LrlDk5DDZFPuvExt789JK)

Table below contains a scaled-down version of a typical distribution
problem faced by a transportation planner at Westvaco’s paper mill in Wickliffe, Kentucky. 

The following table lists cost per unit mile to destination. Note that not all destinations are not travelled by all carriers, so a sparse representation is more desirable for model setup. 

| Destination & Carrier|	ABCT    | IRST    | LAST	  | MRST   | NEST  | PSST	 | 
| ---     | ---   | ---   | ---   | ---  | ---  | ---  | 
|Atlanta      | -	    | 0.88	| 1.15| 0.87 | 0.95 | 1.05 | 
|Everett      | -	| 1.18	| 1.27    | 1.39 |	1.35  | 1.28 | 	
|Ephrata      | -     |	3.42	|1.73	    |1.71   |1.82	    | 2.0 | 
|Riverview	      | 0.79    | 1.01   |1.25   |	0.96| 0.95 |	1.11	   |
|Carson	      | -	| 0.80	|0.87	    | -| 1.0	| -|  
|Chamblee      | -	| 1.23	|1.61  | 1.22 | 1.33	|1.47   |  
|Roseville      | 1.24| 1.13	|1.89	  | 1.32| 1.41	|1.41	   |  
|Hanover      | -	| 4.78	|2.23  | 2.39 | 2.26|2.57	   | 
|Sparks    | -	| 1.45	|-	  | 1.20 | -	|-   |  
|Parsippany      | -	| 1.26|1.36	  | 1.39 | 1.03	|1.76	   |  
|Effingham     | 0.87	| 0.87	|1.25	  | 0.87 | 0.90	|1.31	   | 
|Kearny     | -	| 2.01	|1.54	  | 1.53 | 1.28	|1.95   |

The below table provides the total miles travelled for each trip, Number of trips required and stops in the trip

| Destination |	Trips   | Stops    | Miles	  | 
| ---     | ---   | ---   | ---   | 
|Atlanta      | 4    |0	| 612| 
|Everett      | 1	| 3	| 612    |
|Ephrata      | 3     |	0	|190	    | 
|Riverview	      | 5  | 0   |383   |
|Carson	      | 1	| 2	|3063   |  
|Chamblee      | 1	| 0	|429 |   
|Roseville      | 1| 3	|600  | 
|Hanover      | 1	| 0	|136 |
|Sparks    | 2	| 0	|2439	  |  
|Parsippany      | 1	| 1|355	  |   
|Effingham     | 5	| 0	|570  | 
|Kearny     | 7	| 0	|324	  | 

In addition, the following table shows the minimum per truck charge as well as stop-off charge.

| Rates / Carrier|	ABCT    | IRST    | LAST	  | MRST   | NEST  | PSST	 | 
| ---     | ---   | ---   | ---   | ---  | ---  | ---  | 
|Min charge per truckload: |350	|400	|350	|300	|350	|300      | 
|Stop-off charge:      |50	|75|	50|	35|	50|	50	|

Westvaco wants to determine a minimum-cost shipping plan.
We would like to find a most efficient shipment plan to minimize overall cost.

## Model Formulation

---



### Indices

$i \in \{1..12\}$: Index to represent twelve different destinations

$j \in \{1..6\}$: Index to represent six different carriers

### Parameters

$A$: Set of tuples ($i,j$) where shipment can be made from destination $i$ using carrier $j$.

$s_{i}$: Stops per destination $i$

$t_{i}$: Total trips per destination $i$

$c_{ij}$: Unit shipping cost per mile from destination $i$ using carrier $j$

$m_{i}$: Total miles per destination $i$

$b_{j}$: Minimum charges per carrier $j$

$o_{j}$: Stop-off charges per carrier $j$

$k_{j}$: Commitment per carrier $j$

$p_{j}$: Number of pulls per carrier $j$

### Calculated Parameter
$r_{ij}$: Cost of each trip including miles, stop-off, and minimum charges for destination $i$ using carrier $j$

$r_{ij}= {max}(c_{ij}* m_{i} + o_{j}*s_{i},b_{j})$, $\forall (i,j) \in A$ 

### Decision Variables

$x_{ij}$: Number of trips for destination $i$ using carrier $j$, $(i,j) \in A$.


### Objective Function

- **Cost**. We want to minimize the total cost.


\begin{equation}
\text{Min}_{x_{ij}} \quad \sum_{(i,j) \in A} r_{ij}*x_{ij}
\tag{0}
\end{equation}

### Constraints

\begin{equation}
\sum_{j|(i,j) \in A} x_{ij} \geq t_{i} \quad \forall i \in \{1...12\} \quad (\text{Trips must be completed})
\tag{1}
\end{equation}

\begin{equation}
\sum_{i|(i,j) \in A} x_{ij} \geq k_{j} \quad \forall j \in \{1..6\} \quad (\text{commitment must be satisifed})
\tag{2}
\end{equation}

\begin{equation}
\sum_{i|(i,j) \in A} x_{ij} \leq p_{j} \quad \forall j \in \{1..6\} \quad (\text{must not exceed available pulls})
\tag{3}
\end{equation}

\begin{equation}
x_{ij} \in Integer^+ \quad \forall (i,j) \in A \quad (\text{assignment must be integer values})
\tag{4}
\end{equation}



---

## Python Implementation

We now import the Gurobi Python Module and other Python libraries.

In [ ]:
%pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from itertools import product
from math import sqrt, factorial
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# tested with Gurobi v9.1.0 and Python 3.7.0

Set up the inputs

In [ ]:
#####################################################
#                    Model Formulation
#####################################################

m = gp.Model('carrier selection')

destination = [*range(0,12)]
carrier = [*range(0,6)]

destination_label = ['Atlanta','Everett','Ephrata','Riverview','Carson','Chamblee','Roseville','Hanover','Sparks','Parsippany','Effingham','Kearny']
carrier_label = ['ABCT','IRST','LAST','MRST','NEST','PSST']

# number of stop-offs per destination
s = [0,3,0,0,2,0,3,0,0,1,0,0]

# number of trips required per destination
t = [4,1,3,5,1,1,1,1,2,1,5,7]

# distance in milage required per destination
d = [612,612,190,383,3063,429,600,136,2439,355,570,324]

# minimum charge per carrier
b = [350,400,350,300,350,300]

# stop-off charg per carrier
o = [50,75,50,35,50,50]

# available pulls per carrier
p = [4,8,7,7,3,4]

# committed pulls per carrier
k = [1,7,6,0,0,4]

# unit shipping cost per mile for per destination per carrrier
c = [[0.0,0.88,	1.15,	0.87,	0.95,	1.05],
    [0.0,	1.18,	1.27,	1.39,	1.35,	1.28],
    [0.0,	3.42,	1.73,	1.71,	1.82,	2.0],
    [0.79,	1.01,	1.25,	0.96,	0.95,	1.11],
    [0.0,	0.80,	0.87,	0.0,	1.0,	0.0],
    [0.0,	1.23,	1.61,	1.22,	1.33,	1.47],
    [1.24,	1.13,	1.89,	1.32,	1.41,	1.41],
    [0.0,	4.78,	2.23,	2.39,	2.26,	2.57],
    [0.0,	1.45,	0.0,	1.20,	0.0,	0.0],
    [0.0,	1.62,	1.36,	1.39,	1.03,	1.76],
    [0.87,	0.87,	1.25,	0.87,	0.90,	1.31],
    [0.0,	2.01,	1.54,	1.53,	1.28,	1.95]]



Compute actual charge matrix as well as the set of valid tuples

In [ ]:
# Computing r

# initialize an empty matrix
r = [[0 for j in carrier] for i in destination]

# Valid set of tuples
A = []
for i in destination:
    for j in carrier:
        # if carrier does serve destination
        if c[i][j] > 0:
            # compute actual charge, taking into account minimum charge
            r[i][j] = max(d[i]*c[i][j] + o[j]*s[i],b[j])

            # build valid set of tuples
            tp = i,j
            A.append(tp)

print(np.matrix(r))    

# valid set of routes that can be covered by each carrier (denote as j)
AJ = [] 
n = 0
for l in carrier:
    A_temp = []
    for i in destination:
        for j in carrier:
            if r[i][j] > 0:
                if j==n:
                    tp = i,j
                    A_temp.append(tp)
    AJ.append(A_temp) 
    n+=1               

#print(np.matrix(AJ[0]))

# valid set of carriers that can cover each destination (denote as i)

AI = [] 
for n in range(len(destination)):
    A_temp = [(i,j) for i in destination for j in carrier if r[i][j] > 0 if i==n]
    AI.append(A_temp)

#print(np.matrix(AI[0]))


[[   0.    538.56  703.8   532.44  581.4   642.6 ]
 [   0.    947.16  927.24  955.68  976.2   933.36]
 [   0.    649.8   350.    324.9   350.    380.  ]
 [ 350.    400.    478.75  367.68  363.85  425.13]
 [   0.   2600.4  2764.81    0.   3163.      0.  ]
 [   0.    527.67  690.69  523.38  570.57  630.63]
 [ 894.    903.   1284.    897.    996.    996.  ]
 [   0.    650.08  350.    325.04  350.    349.52]
 [   0.   3536.55    0.   2926.8     0.      0.  ]
 [   0.    650.1   532.8   528.45  415.65  674.8 ]
 [ 495.9   495.9   712.5   495.9   513.    746.7 ]
 [   0.    651.24  498.96  495.72  414.72  631.8 ]]


Setup decisions, objective, and constraints

In [ ]:
# Build decision variables: whether to assign destination i to carrier j
x = m.addVars(A, vtype=GRB.CONTINUOUS, name='Assign')

In [ ]:
# Objective function: Minimize total cost
m.setObjective(gp.quicksum(r[i][j]*x[(i,j)] for i,j in A), GRB.MINIMIZE)

In [ ]:
#Constraints

# Commitment Constraints
ComConstrs = m.addConstrs((gp.quicksum(x[(i,j)] for i,j in AJ[j]) >= k[j] for j in carrier), 
                                      name='CommitmentConstrs')

# Available Pull Constraints
PulConstrs = m.addConstrs((gp.quicksum(x[(i,j)] for i,j in AJ[j]) <= p[j] for j in carrier), 
                                      name='PullConstrs')

#Trip Constraints
TriConstrs = m.addConstrs((gp.quicksum(x[(i,j)] for i,j in AI[i]) >= t[i] for i in destination),
                                      name='TripConstrs')

Solve the model

In [ ]:
# Run optimization engine
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 24 rows, 58 columns and 174 nonzeros
Model fingerprint: 0x86c2a906
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+02, 4e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 8e+00]
Presolve removed 6 rows and 0 columns
Presolve time: 0.02s
Presolved: 18 rows, 61 columns, 119 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.500000e+01   0.000000e+00      0s
      24    2.2394380e+04   0.000000e+00   0.000000e+00      0s

Solved in 24 iterations and 0.03 seconds (0.00 work units)
Optimal objective  2.239438000e+04


Examine outputs

In [ ]:
# check the optimal cost

print("The total cost for the carrier selection would be $",m.ObjVal)

The total cost for the carrier selection would be $ 22394.38


In [ ]:
# print optimal assignment by destinations

print("\033[1m Optimal assignment by destinations")
print("------------------------------------------\n")
# loop through all destinations
for i in destination:
  print("\033[1m",destination_label[i],"\033[0m (requires",t[i],"trips): \n", end="    ")

  # check whether a carrier serves the destination
  for j in carrier:
    
    # if carrier j serves destination i
    if (i,j) in A and abs(x[(i,j)].x) > 1e-6:
      print(carrier_label[j], "does", x[i,j].x, "trips", end=", ")
  print("\n")  


 Optimal assignment by destinations
------------------------------------------

 Atlanta  (requires 4 trips): 
    MRST does 4.0 trips, 

 Everett  (requires 1 trips): 
    PSST does 1.0 trips, 

 Ephrata  (requires 3 trips): 
    LAST does 1.0 trips, PSST does 2.0 trips, 

 Riverview  (requires 5 trips): 
    ABCT does 4.0 trips, MRST does 1.0 trips, 

 Carson  (requires 1 trips): 
    IRST does 1.0 trips, 

 Chamblee  (requires 1 trips): 
    IRST does 1.0 trips, 

 Roseville  (requires 1 trips): 
    IRST does 1.0 trips, 

 Hanover  (requires 1 trips): 
    PSST does 1.0 trips, 

 Sparks  (requires 2 trips): 
    MRST does 2.0 trips, 

 Parsippany  (requires 1 trips): 
    NEST does 1.0 trips, 

 Effingham  (requires 5 trips): 
    IRST does 5.0 trips, 

 Kearny  (requires 7 trips): 
    LAST does 5.0 trips, NEST does 2.0 trips, 



In [ ]:
# print optimal assignment by carriers

print("\033[1m Optimal assignment by carriers")
print("------------------------------------------\n")
# loop through all carriers
for j in carrier:
  print("\033[1m",carrier_label[j],"\033[0m (",k[j],"committed and",p[j],"available pulls): \n Serves ", end="")

  # check whether a carrier serves the destination
  sum_trip = 0
  for i in destination:
    
    # if carrier j serves destination i
    if (i,j) in A and abs(x[(i,j)].x) > 1e-6:
      print(x[i,j].x, "trips to", destination_label[i], end=", ")
      sum_trip+=x[i,j].x
  print("\n Total:",sum_trip,"trips\n")  


 Optimal assignment by carriers
------------------------------------------

 ABCT  ( 1 committed and 4 available pulls): 
 Serves 4.0 trips to Riverview, 
 Total: 4.0 trips

 IRST  ( 7 committed and 8 available pulls): 
 Serves 1.0 trips to Carson, 1.0 trips to Chamblee, 1.0 trips to Roseville, 5.0 trips to Effingham, 
 Total: 8.0 trips

 LAST  ( 6 committed and 7 available pulls): 
 Serves 1.0 trips to Ephrata, 5.0 trips to Kearny, 
 Total: 6.0 trips

 MRST  ( 0 committed and 7 available pulls): 
 Serves 4.0 trips to Atlanta, 1.0 trips to Riverview, 2.0 trips to Sparks, 
 Total: 7.0 trips

 NEST  ( 0 committed and 3 available pulls): 
 Serves 1.0 trips to Parsippany, 2.0 trips to Kearny, 
 Total: 3.0 trips

 PSST  ( 4 committed and 4 available pulls): 
 Serves 1.0 trips to Everett, 2.0 trips to Ephrata, 1.0 trips to Hanover, 
 Total: 4.0 trips



#Conclusion

The least cost assignment of truck loads to carriers that meets the necessary requirements is found through the above integer programming model. The minimum cost is $22,394. 

A key take away of the above example is that the minimum charges can be pre-processed in the input section, so that the model works directly with the correct charges with minimum charge already incorprated. Further, since not every carrier will serve all the routes, we use a sparse representation, i.e., a set of valid tuples of carrier-destination pair, to setup the model. This sparse representation is more efficient than utilizing large numbers for destinations that a carrier does not serve. 

##  References

[1] Gurobi python reference. https://www.gurobi.com/documentation/

[2] This notebook is contributed by Manoj Kumar Rayana, * Shri Lekha Kasulu Pramod Kumar, Irene Issac Dapril, Akarsh Jayachamarajapura Devarajaiah, and
Shengming Ye